In [ ]:
!pip install --quiet langchain
!pip install --quiet pinecone
!pip install --quiet sentence_transformers
!pip install --quiet pinecone-client
!pip install --quiet google-generativeai
print('Done Installation.')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 4.4 MB/s eta 0:00:00
Done Installation.


In [ ]:
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.2 MB/s eta 0:00:00


In [ ]:
import re
import pandas as pd
from io import BytesIO
from typing import Tuple, List
import pickle
import csv
import os
import json


import pinecone
from pinecone import Pinecone, ServerlessSpec
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings


from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

import google.generativeai as genai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
product_data = pd.read_csv('/content/drive/MyDrive/products.csv')

In [ ]:
product_data.head(2)

,Name,Variant Name,Price,Size,Material,Compatibility,Features,URL,Meta Title,Meta Description,Product Description
0,iPhone 7 Series - Hybrid Armor,iPhone 7 Plus / Black / In Stock,39.99,"['Dimensions: 6.46 x 3.28 x 0.44 inches', 'Pro...","['TPU: Thermoplastic Polyurethane', 'PC: Polyc...",['Apple - iPhone 7 Plus'],['New generation of dual-layered protection fo...,https://www.spigen.com/products/iphone-7-serie...,iPhone 7 Series Hybrid Armor Case - Spigen.com...,Protect your iPhone 7 Plus with one of our tou...,\n\n\n\n\n\nMore than just protection.\nExplor...
1,Tesla Model 3 - Armrest Console Organizer | TO...,Black / In Stock,39.99,['Package Weight: 12.5 oz'],['PC: Polycarbonate'],['Tesla - Model 3 (2024) Highland'],"['Made from PC for durability', 'Precise fit i...",https://www.spigen.com/products/tesla-model-3-...,Tesla Model 3 Armrest Console Organizer - Spig...,Keep your Tesla Model 3 organized. Shop our Ar...,\n\n\n\n\n\nMore than just protection.\nExplor...


In [ ]:
# converting csv to json

csv_file_path = '/content/drive/MyDrive/products.csv'
json_file_path = '/content/drive/MyDrive/products_data.json'

if not os.path.exists(json_file_path):
    with open(csv_file_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        data = []

        for row in reader:
            data_object = {
                "Name": row["Name"],
                "Variant Name": row["Variant Name"],
                "Price": row["Price"],
                "Size": row["Size"],
                "Material": row["Material"],
                "Compatibility": row["Compatibility"],
                "Features": row["Features"],
                "URL": row["URL"],
                "Meta Title": row["Meta Title"],
                "Meta Description": row["Meta Description"],
                "Product Description": row["Product Description"]
            }

            data.append(data_object)

    json_data = json.dumps(data, indent=4)

    with open(json_file_path, 'w') as jsonfile:
        jsonfile.write(json_data)
else:
    print("JSON file already exists.")

JSON file already exists.


In [ ]:
# Splitting json file

def split_json_to_files(json_file_path, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with open(json_file_path, 'r') as json_file:
        products = json.load(json_file)


    for i, product in enumerate(products):
        product_details = {
            'id': i,
            "Name": product["Name"],
            "Variant Name": product["Variant Name"],
            "Price": product["Price"],
            "Size": product["Size"],
            "Material": product["Material"],
            "Compatibility": product["Compatibility"],
            "Features": product["Features"],
            "URL": product["URL"],
            "Meta Title": product["Meta Title"],
            "Meta Description": product["Meta Description"],
            "Product Description": product["Product Description"]
        }

        output_file_path = os.path.join(output_dir, f'{i}.json')
        with open(output_file_path, 'w') as output_file:
            json.dump(product_details, output_file, indent=4)

        #print(f"Saved details of product {i} to {output_file_path}")


In [ ]:
output_dir = '/content/drive/MyDrive/Prod_one_each'
json_file_path = '/content/drive/MyDrive/products_data.json'
split_json_to_files(json_file_path, output_dir)

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Get embeddings

def embed_product_information(json_file_path):
    with open(json_file_path, 'r') as json_file:
        product_data = json.load(json_file)

    text_attributes = [
        product_data['Name'],
        product_data['Variant Name'],
        product_data['Price'],
        ', '.join(eval(product_data['Size'])) if product_data['Size'] != 'NA' else '',
        ', '.join(eval(product_data['Material'])) if product_data['Material'] != 'NA' else '',
        ', '.join(eval(product_data['Compatibility'])) if product_data['Compatibility'] != 'NA' else '',
        ', '.join(eval(product_data['Features'])) if product_data['Features'] != 'NA' else '',
            product_data['Meta Title'],
        product_data['Meta Description'],
        product_data['Product Description']
    ]
    concatenated_text = '\n'.join(text_attributes)
    concatenated_text_with_image_url = concatenated_text + '\n' + product_data['URL']

    model_name = 'sentence-transformers/all-MiniLM-L6-v2'
    embeddings_model = SentenceTransformer(model_name)
    embedding = embeddings_model.encode(concatenated_text_with_image_url )

    return embedding


In [ ]:
def process_json_files(directory):
    embeddings_data = []

    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            json_file_path = os.path.join(directory, filename)
            embedding = embed_product_information(json_file_path)

            with open(json_file_path, 'r') as json_file:
              product_data = json.load(json_file)

            product_id = product_data['id']
            product_name = product_data['Name']
            embeddings_data.append({'id': product_id, 'name': product_name, 'embedding': embedding.tolist()})

            # Save embedding data to a separate JSON file for each product
            output_json_path = os.path.join('/content/drive/MyDrive/embeddings', f'embedding_{product_id}.json')
            with open(output_json_path, 'w') as output_json_file:
                json.dump({'id': product_id, 'name': product_name, 'embedding': embedding.tolist()}, output_json_file)

    return embeddings_data



In [ ]:
# Directory containing JSON files
directory_path = '/content/drive/MyDrive/Prod_one_each'

# Process JSON files in the directory
embeddings_data = process_json_files(directory_path)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
with open('all_embeddings.json', 'w') as all_embeddings_file:
    json.dump(embeddings_data, all_embeddings_file)

In [ ]:
json_embed_path = "all_embeddings.json"

with open(json_embed_path, 'r') as json_file:
    embedded_product_data = json.load(json_file)

# Find the maximum length of a single embedding
max_embedding_length = max(len(embed["embedding"]) for embed in embedded_product_data)

print("Maximum length of a single embedding:", max_embedding_length)


Maximum length of a single embedding: 384


In [ ]:
#Connect with pinecone

pc = Pinecone(
        api_key='0ea3ebe6-7fc2-4312-9033-2f979891f7e'
    )

if 'productdb' not in pc.list_indexes().names():
        pc.create_index(
            name='productdb',
            dimension= 384,
            metric='cosine',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-east-1'
            )
        )

In [ ]:
from pinecone import Index

In [ ]:
#upserting into the database
# Set your index name (create it in Pinecone beforehand)
index_name = "productdb"


def store_product_embeddings(embeddings_dir):
    index = Index(api_key='0ea3ebe6-7fc2-4312-9033-2f979891f7e', host ='https://productdb-hwxvhuf.svc.aped-4627-b74a.pinecone.io')

    for filename in os.listdir(embeddings_dir):
        if not filename.endswith(".json"):
            continue  # Ignore non-JSON files

        # Load product information
        with open(os.path.join(embeddings_dir, filename), "r") as f:
            product_data = json.load(f)

        # Extract product ID, name, and embedding from JSON data
        product_id = product_data.get("id")  # Use get() for safer access
        #product_name = product_data.get("name")  # Use get() for safer access
        embedding = product_data.get("embedding")  # Use get() for safer access

        if not all([product_id, embedding]):  # Check for missing data
            print(f"Warning: Skipping {filename} due to missing data.")
            continue

        # Combine product information and embedding into a single dictionary
        vector_data = {

                "id": str(product_id),  # Use product ID as embedding ID (optional)
                "values": embedding,
        }

        # Upsert data into Pinecone (using product ID as primary identifier)
        index.upsert([vector_data], index_name)

    print("Product information and embeddings stored successfully!")


In [ ]:
# Example usage
embeddings_dir = "/content/drive/MyDrive/embeddings"  # Update with your path
store_product_embeddings(embeddings_dir)

Product information and embeddings stored successfully!


In [ ]:
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
embeddings_model = SentenceTransformer(model_name)

In [ ]:
query = "apple phone cover"
query_embeddings = embeddings_model.encode(query)
query_embeddings = query_embeddings.tolist()
print(query_embeddings)

[-0.11660333722829819, 0.1270860731601715, 0.12138770520687103, 0.03067575953900814, 0.0725066140294075, 0.004728072322905064, 0.08785577118396759, 0.06143241003155708, 0.03887192904949188, 0.0050056092441082, 0.00038831387064419687, 0.02804035134613514, 0.021794432774186134, -0.014340441673994064, 0.010852686129510403, -0.06293728947639465, -0.03630442172288895, -0.09061874449253082, -0.06224623695015907, 0.014907819218933582, -0.04189539700746536, 0.024043504148721695, 0.046838290989398956, -0.05436097830533981, -0.031049678102135658, 0.03940298780798912, 0.02017873525619507, 0.004445951897650957, -0.05317278951406479, -0.030207762494683266, -0.018874289467930794, 0.03221960365772247, 0.07780658453702927, 0.12149076163768768, -0.01258745975792408, -0.03978922218084335, -0.029575513675808907, 0.09845577925443649, -0.0521225705742836, 0.05894491821527481, -0.05219601094722748, 0.030880404636263847, -0.008696917444467545, 0.06430858373641968, 0.005726141855120659, 0.03567105904221535, -

In [ ]:
index = Index(api_key='0ea3ebe6-7fc2-4312-9033-2f979891f7e', host ='https://productdb-hwxvhuf.svc.aped-4627-b74a.pinecone.io')


In [ ]:
results = index.query(
    index_name='productdb',
    vector= query_embeddings,
    top_k = 10
)

In [ ]:
results= index.query(
    namespace="productdb",
    vector= query_embeddings,
    top_k=10,
    include_values=True,
    include_metadata=True,
    #filter={"genre": {"$eq": "action"}}
)

In [ ]:
print(results["matches"])

[{'id': '1424',
 'score': 0.622525692,
 'values': [-0.127961829,
            0.0705556497,
            0.0905874297,
            -0.0254514758,
            0.0107620349,
            -0.00663976138,
            0.0752189606,
            0.0574353524,
            -0.0287975445,
            -0.0314770192,
            0.0735988319,
            0.01865777,
            0.0135083375,
            0.0203694142,
            -0.0311933849,
            -0.0215723142,
            -0.00171045342,
            -0.0907328948,
            0.0151209887,
            0.0346411578,
            -0.0243646745,
            -0.0308047459,
            0.010774863,
            -0.000808095909,
            -0.0921234339,
            0.0105372537,
            -0.0094695352,
            -0.0150120612,
            -0.0109533593,
            -0.0781276748,
            0.000594554876,
            0.00970912073,
            0.068776,
            0.0690309256,
            -0.0146826245,
            -0.0974353254,
       

In [ ]:
for result in results["matches"]:
    product_id = result["id"]  # Assuming "id" field exists in embedding
    similarity_score = result["score"]
    prod_embed = result["values"]
    print(f"Product ID: {product_id}, Similarity Score: {similarity_score}")

Product ID: 1424, Similarity Score: 0.622525692
Product ID: 89, Similarity Score: 0.604651511
Product ID: 456, Similarity Score: 0.602901816
Product ID: 457, Similarity Score: 0.602121055
Product ID: 459, Similarity Score: 0.601001382
Product ID: 458, Similarity Score: 0.600600243
Product ID: 2548, Similarity Score: 0.600478351
Product ID: 1425, Similarity Score: 0.599473357
Product ID: 2549, Similarity Score: 0.595885
Product ID: 2547, Similarity Score: 0.593785167


In [ ]:
#configuration of gemini AI
gemini_api_key= API_Key
genai.configure(api_key = gemini_api_key)
model = genai.GenerativeModel("gemini-pro")

**Getting the Prompt style ready**

In [ ]:
def get_system_prompt():
  return "Hi There! Welcome back to Spigen. Are you looking for a new case for your recent phone purchase, or perhaps some screen protectors?"

In [ ]:
#executing the system prompt firstly

sys_prompt = get_system_prompt()
print(sys_prompt)

Hi There! Welcome back to Spigen. Are you looking for a new case for your recent phone purchase, or perhaps some screen protectors?


In [ ]:
def generate_response(prompt):
    """Sends prompt to Gemini and returns the generated response."""
    response = model.generate_content(prompt)
    return response.text

# Example usage
user_input = "What is the capital of France?"
response = generate_response(user_input)
print(f"User: {user_input}")
print(f"AI: {response}")

User: What is the capital of France?
AI: Paris


In [ ]:
user_query = "What's the best Spigen case for the iPhone 13 Pro Max?"

In [ ]:
user_embedding = embeddings_model.encode(user_query)
user_embedding = user_embedding.tolist()

In [ ]:
results= index.query(
    namespace="productdb",
    vector= user_embedding,
    top_k=10,
    include_values=True,
    include_metadata=True,
    #filter={"genre": {"$eq": "action"}}
)

In [ ]:
for result in results["matches"]:
    product_id = result["id"]  # Assuming "id" field exists in embedding
    similarity_score = result["score"]
    prod_embed = result["values"]
    print(f"Product ID: {product_id}, Similarity Score: {similarity_score}")

Product ID: 1566, Similarity Score: 0.75479728
Product ID: 1574, Similarity Score: 0.753170967
Product ID: 1564, Similarity Score: 0.752819836
Product ID: 1782, Similarity Score: 0.752016544
Product ID: 615, Similarity Score: 0.751612246
Product ID: 1569, Similarity Score: 0.751606584
Product ID: 1570, Similarity Score: 0.751454949
Product ID: 1783, Similarity Score: 0.749988735
Product ID: 1577, Similarity Score: 0.749281406
Product ID: 1565, Similarity Score: 0.749077499


In [ ]:
data_path_search = '/content/drive/MyDrive/Prod_one_each'

In [ ]:
def get_info_search_prod(results):
  Relevant_Info = []

  for result in results['matches']:
    product_id = result['id']
    prod_filename = f'{product_id}.json'


    #File_path for json file
    file_path = os.path.join(data_path_search, prod_filename)

    with open(file_path, 'r') as prod_file:
      prod_data = json.load(prod_file)

    #Getting product info
    prod_Info = {
            "Name": prod_data["Name"],
            "Variant Name": prod_data["Variant Name"],
            "Price": prod_data["Price"],
            "Size": prod_data["Size"],
            "Material": prod_data["Material"],
            "Compatibility": prod_data["Compatibility"],
            "Features": prod_data["Features"],
            "URL": prod_data["URL"],
            "Meta Title": prod_data["Meta Title"],
            "Meta Description": prod_data["Meta Description"],
            "Product Description": prod_data["Product Description"]
        }
    Relevant_Info.append(prod_Info)

  return Relevant_Info

#------------------------------------------------------------------------------
def get_formatted_response(user_query, relevant_info):

  response_template = "Here's some information about products that might be relevant to your query \"{}\":\n\n".format(user_query)

  for product in relevant_info:
    product_info = """* **{}**\n""".format(product["Name"])
    product_info += "  * Color: {}\n".format(product.get("Color", "N/A"))
    product_info += "  * Compatibility: {}\n".format(product["Compatibility"])
    product_info += "  * Price: {}\n".format(product.get("Price", "N/A"))

    # Handle potentially long descriptions with truncation
    description = product.get("Meta Description", "N/A")+ "..." if product.get("Meta Description")  else "N/A"
    product_info += "  * Description: {}\n".format(description)

    #product_info += "  * URL: {}\n\n".format(product.get("URL", "N/A"))
    response_template += product_info

  response_template += "Let me know if you have any other questions!"

  print(f"AI Response: {response_template}")

In [ ]:
Relevant_Info = get_info_search_prod(results)
get_formatted_response(user_query, Relevant_Info)

AI Response: Here's some information about products that might be relevant to your query "What's the best Spigen case for the iPhone 13 Pro Max?":

* **iPhone 12 Series - Ultra Hybrid**
  * Color: N/A
  * Compatibility: ['Apple - iPhone 12 Pro Max']
  * Price: 29.99
  * Description: Showcase your iPhone 12 / Mini / Pro / Pro Max in one of our clear protective cases. The Ultra Hybrid case is perfect for the minimalist seeking a slim case with added protection, while enjoying the look of their original device. Shop now....
* **iPhone 12 Series - Ultra Hybrid**
  * Color: N/A
  * Compatibility: ['Apple - iPhone 12 Pro Max']
  * Price: 29.99
  * Description: Showcase your iPhone 12 / Mini / Pro / Pro Max in one of our clear protective cases. The Ultra Hybrid case is perfect for the minimalist seeking a slim case with added protection, while enjoying the look of their original device. Shop now....
* **iPhone 12 Series - Ultra Hybrid**
  * Color: N/A
  * Compatibility: ['Apple - iPhone 12 Pr